In [31]:
from qiime2 import Artifact
import tempfile
import os
import pandas as pd

In [6]:
working_dir = os.path.abspath('../')

In [ ]:
#what are the raw DNA sequences annotated by extended taxonomies
#as mitochondria, that are not annotated as such by the base taxonomy?
base_refs=['greengenes', 'silva']
for base in base_refs:
    organelle='Mitochondria' if base=='silva' else 'mitochondria' 
    base_path=working_dir+'/output/'+base+'_classification.qza'
    base_classification=Artifact.load(base_path)
    m2_path=working_dir+'/output/'+base+'_metaxa2_classification.qza'
    m2_classification=Artifact.load(m2_path)
    with tempfile.TemporaryDirectory() as temp_dir:
        m2_classification.export_data(temp_dir)
        m2_taxonomy=pd.read_csv(temp_dir+'/taxonomy.tsv', '\t')
    #this is maybe bad practice to have so many commands in the same line.
    #finds every row with 'mitochondria' in the 'Taxon' column and adds the
    #sequence to a list
    mito_seqs_m2=m2_taxonomy[m2_taxonomy['Taxon'].str.contains(organelle)]['Feature ID'].tolist()
    with tempfile.TemporaryDirectory() as temp_dir:
        base_classification.export_data(temp_dir)
        base_taxonomy=pd.read_csv(temp_dir+'/taxonomy.tsv', '\t')
    mito_seqs_base=base_taxonomy[base_taxonomy['Taxon'].str.contains(organelle)]['Feature ID'].tolist()
    diff_seqs=list(set(mito_seqs_m2)-set(mito_seqs_base))
    with open(working_dir+'/output/'+base+'_differentially_annotated_sequences.txt', 'a') as file:
        file.writelines('>%s\n%s\n' % (seq, seq) for seq in diff_seqs)

In [30]:
#rewrite the above such that it shows me all sequences annotated differently, not just mitochondria
base_refs=['greengenes', 'silva']
for base in base_refs:
    base_path = working_dir+'/output/'+base+'_reference_taxonomy.qza'
    base_classification = Artifact.load(base_path)
    with tempfile.TemporaryDirectory() as temp_dir:
        base_classification.export_data(temp_dir)
        base_taxonomy=pd.read_csv(temp_dir+'/taxonomy.tsv', '\t')
    m2_path = working_dir+'/output/'+base+'_metaxa2_reference_taxonomy.qza'
    m2_classification=Artifact.load(m2_path)
    with tempfile.TemporaryDirectory() as temp_dir:
        m2_classification.export_data(temp_dir)
        m2_taxonomy=pd.read_csv(temp_dir+'/taxonomy.tsv', '\t')
    base_taxonomy = base_taxonomy.sort_values('Feature ID')
    base_taxonomy = base_taxonomy.reset_index()
    base_taxonomy = base_taxonomy.drop(['Consensus','index'],1)
    m2_taxonomy = m2_taxonomy.sort_values('Feature ID')
    m2_taxonomy = m2_taxonomy.reset_index()
    m2_taxonomy = m2_taxonomy.drop(['Consensus','index'],1)
    results = base_taxonomy.compare(m2_taxonomy)
    results.columns = ['base', 'extended']
    results['sequence'] = base_taxonomy['Feature ID']
    with open(base+'_differential_annotations.fasta', 'w') as fasta_file:
        for row in results.itertuples(False, None):
            fasta_file.write('>'+row[0]+'|'+row[1]+'\n'+row[2]+'\n')
    with open(base+'_differential_annotations.tsv', 'w') as tsv:
        tsv.write('sequence\t'+base+'_annotation\textended_annotation\n')
        for row in results.itertuples(False, None):
            tsv.write(row[2]+'\t'+row[0]+'\t'+row[1]+'\n')